In [5]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import pdb
import os
os.chdir("/data/tianyu_guo/birth")

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple

from ihead_data import DataArgs, Dataset, iterate_batches
from ihead_full_model import ModelArgs, Transformer
%load_ext autoreload
%autoreload 2

In [2]:
cfg = OmegaConf.load("/data/tianyu_guo/birth/test_run/params.yaml")
model = Transformer(cfg.model_args)

In [3]:
model.eval().cuda()

Transformer(
  (tok_embeddings): Embedding(65, 256)
  (pos_embeddings): Embedding(256, 256)
  (layers): ModuleList(
    (0-3): 4 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=256, out_features=256, bias=False)
        (wk): Linear(in_features=256, out_features=256, bias=False)
        (wv): Linear(in_features=256, out_features=256, bias=False)
        (wo): Linear(in_features=256, out_features=256, bias=False)
      )
      (ff): FeedForward(
        (w1): Linear(in_features=256, out_features=1024, bias=False)
        (w2): Linear(in_features=1024, out_features=256, bias=False)
      )
      (attention_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ff_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (output): Linear(in_features=256, out_features=65, bias=False)
)

In [4]:
ds = Dataset(cfg.data_args, train_test=None)
ds_test = Dataset(cfg.data_args, train_test=None)
ds_test.idxs = ds.idxs
cfg.model_args.vocab_size = ds.num_tokens


In [7]:
for i, (x, y, outs) in enumerate(iterate_batches(ds, batch_size=cfg.optim_args.batch_size,
                                                     num_workers=cfg.num_data_workers)):
    x = torch.from_numpy(x).cuda()
    y = torch.from_numpy(y).cuda()
    outs = torch.from_numpy(outs).cuda()
    pred = model(x)

> /data/tianyu_guo/birth/ihead_full_model.py(82)forward()
     80         xq, xk, xv = xq.transpose(1, 2), xk.transpose(1, 2), xv.transpose(1, 2)
     81         pdb.set_trace()
---> 82         scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
     83         scores = scores + mask  # (bs, n_heads, slen, slen)
     84         scores = F.softmax(scores.float(), dim=-1).type_as(x)

> /data/tianyu_guo/birth/ihead_full_model.py(83)forward()
     81         pdb.set_trace()
     82         scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
---> 83         scores = scores + mask  # (bs, n_heads, slen, slen)
     84         scores = F.softmax(scores.float(), dim=-1).type_as(x)
     85         output = torch.matmul(scores, xv)  # (bs, n_heads, slen, head_dim)

tensor([[ 0.1153, -0.3941,  0.3326,  0.1721,  0.2070,  0.2423,  0.4870,  0.6502,
          0.1720,  0.2510],
        [ 0.0370,  0.0156,  0.5328,  0.2182,  0.1195,  0.3538,  0.2228,  0.3328,
   